In [1]:
using Pkg
Pkg.activate(".")

include("src/rainforest_lib.jl")
using .Rainforestlib

  Activating project at `~/workspace/uni/spatial_temporal_data/spatio_temporal_data-rainforest_development`
[ Info: Precompiling Zarr [0a941bbe-ad1d-11e8-39d9-ab76183a1d99]
[ Info: Precompiling YAXArrays [c21b50f5-aa40-41ea-b809-c0f5e47bfa5c]
[ Info: Precompiling GeoMakie [db073c08-6b98-4ee5-b6a4-5efafb3259c6]
[ Info: Precompiling GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a]


In [2]:
using Zarr
using YAXArrays

lc = Rainforestlib.get_lcc_datacube()

filtered_dc = Rainforestlib.rough_spatial_filter(lc; time_bounds = (Date(2010), Date(2021)))

small_soy_region_dc = Rainforestlib.rough_spatial_filter(
    lc;
    lon_bounds = (-57, -54),
    lat_bounds = (-10, -6), 
    time_bounds = (Date(2010), Date(2021))
)

YAXArray with the following dimensions
lon                 Axis with 1080 Elements from -56.998611111111096 to -54.001388888888876
lat                 Axis with 1440 Elements from -6.001388888888883 to -9.998611111111105
time                Axis with 11 Elements from 2010-01-01T00:00:00 to 2020-01-01T00:00:00
name: lccs_class
Total size: 16.31 MB


In [12]:


# set a set of accepted values
rainforest_values = Set(Rainforestlib.LCCSClasses.categories["rainforest"].lccs_classes)

forest_values = Set(Rainforestlib.LCCSClasses.categories["forest"].lccs_classes)

all_forest_values = union(forest_values, rainforest_values)

Set{String} with 13 elements:
  "tree_needleleaved_evergreen_open"
  "tree_needleleaved_deciduous_closed"
  "tree_needleleaved_deciduous_open"
  "tree_needleleaved_deciduous_closed_to_open"
  "tree_broadleaved_deciduous_open"
  "tree_broadleaved_deciduous_closed"
  "tree_broadleaved_evergreen_closed_to_open"
  "tree_cover_flooded_saline_water"
  "tree_needleleaved_evergreen_closed"
  "tree_mixed"
  "tree_broadleaved_deciduous_closed_to_open"
  "tree_cover_flooded_fresh_or_brakish_water"
  "tree_needleleaved_evergreen_closed_to_open"

In [42]:
function count_replacement(old_flag_matrix::Matrix{UInt8}, new_flag_matrix::Matrix{UInt8}, tracked_category::LCCSClasses.Category)::Dict{UInt8, Int64}
    # it is assumed that both have the same size

    result = Dict()

    (rows, cols) = size(old_flag_matrix)
    for row in 1:rows
        for col in 1:cols
            old_value = old_flag_matrix[row, col]
            new_value = new_flag_matrix[row, col]

            if old_value != new_value && old_value in tracked_category.lccs_flags 

                if new_value in keys(result)
                    result[new_value] += 1
                else
                    result[new_value] = 1
                end
            end
        end
    end

    return result

end

count_replacement (generic function with 2 methods)

In [46]:
function get_replacement_data(
    datacube,
    timestep::Int,
    tracked_category::Rainforestlib.LCCSClasses.Category
)::Tuple{Array{Int}, Array{Int}, Array{Int}}

    timesteps = YAXArrays.getAxis("time", datacube).values

    # step number is aways one smaller then the start
    step = timestep - 1

    old_year = timesteps[timestep-1]
    new_year = timesteps[timestep]

    results_dict = count_replacement(datacube[:, :, timestep-1], datacube[:, :, timestep], tracked_category)

    sorted_result_keys = sort!([k for k in keys(results_dict)])


    x_vals = fill(step, length(sorted_result_keys))

    y_vals = [results_dict[k] for k in sorted_result_keys]

    grp = 1:length(sorted_result_keys)


    return (x_vals, y_vals, grp)

end


get_replacement_data (generic function with 2 methods)

In [47]:
function build_replacement_figure(
    datacube,
    tracked_category::LCCSClasses.Category;
)

    timesteps = YAXArrays.getAxis("time", datacube).values

    x_vals = Vector()
    y_vals = Vector()
    gpr_vals = Vector()

    xaxis_names = [string(timesteps[timestep-1]) * " to " * string(timesteps[timestep]) for timestep in 2:length(timesteps)]

    for timestep in 2:length(timesteps)

        old_year = timesteps[timestep-1]
        new_year = timesteps[timestep]

        results_dict = count_replacement(datacube[:, :, timestep-1], datacube[:, :, timestep], tracked_category)

        x, y, gpr = get_replacement_data(
            datacube,
            timestep,
            tracked_category
        )

        println("X: $(x); Y: $(y); GPR: $(gpr)")

        x_vals = [x_vals; x]
        y_vals = [x_vals; y]
        gpr_vals = [gpr_vals; gpr]
        
    end    

    println("\n\n\nX: $(x_vals); Y: $(y_vals); GPR: $(gpr_vals)")
    println(xaxis_names)
end

build_replacement_figure (generic function with 1 method)

In [48]:
build_replacement_figure(filtered_dc, LCCSClasses.categories["rainforest"])

LoadError: MethodError: no method matching sort(::Base.KeySet{UInt8, Dict{UInt8, Int64}})

[0mClosest candidates are:
[0m  sort([91m::AbstractUnitRange[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:1384[24m[39m
[0m  sort([91m::AbstractRange[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:1387[24m[39m
[0m  sort([91m::OrderedCollections.LittleDict[39m; byvalue, args...)
[0m[90m   @[39m [36mOrderedCollections[39m [90m~/.julia/packages/OrderedCollections/vTcqn/src/[39m[90m[4mdict_sorting.jl:38[24m[39m
[0m  ...
